## Demonstrate Various scene module functions

In [ ]:
include("../modules/geometry.jl")
include("../modules/scene.jl")
using Plots
pyplot()

## planetary shape constants
earth_radius=6378.137e3
earth_eccentricity=sqrt(0.00669437999015)

# target volume grid on surface defined in geodetic lat, lon, height (θϕh)
t_θ=-90:2:90 # deg
t_ϕ=-180:4:180 # deg
t_h=0:100:9000 # m target heights

#try the two different methods for creating this
@time t_geo_grid=Scene.form3Dgrid_for(t_θ,t_ϕ,t_h) # using 3 nested for loops 
@time t_geo_grid=Scene.form3Dgrid_array(t_θ,t_ϕ,t_h) # using array processing

#convert lat,lon,height grid to ECEF-xyz
t_xyz_grid=Geometry.geo_to_xyz(t_geo_grid) # convert target volume from geo to xyz

# plot the target grid 
scatter(t_xyz_grid[1,:],t_xyz_grid[2,:],t_xyz_grid[3,:],camera=(0,0),
        markersize=0.1)#display grid in 3D


## Define target volume using relative azimuth, elevation and height

In [ ]:

#define target-grid limits
el  = 0:1:30 # elevation angles (deg)
az  = -180:10:180 # azimuth angles (deg)
hgt = 0:10:200 # target heights (m)

#create grid based on az,el,hgt
@time target_grid = Scene.form3Dgrid_for(az,el,hgt) # using 3 nested for loops

#platform location in geodetic coordinates
platform_geo = [10,50,6000e3] # platform latitude, longitude, altitude [deg,deg,m]

#create peg point
γ   = 0 # deg track angle (heading), 0 deg is north, 90 deg is east
peg = Geometry.PegPoint(platform_geo[1],platform_geo[2],γ) # deg peg point is the nadir point of platform at the center of SAR aperture

ecef_target_grid = Scene.lookh_to_xyz(target_grid,platform_geo,peg) # convert target volume from relative look vector to xyz

#display 3D grid
scatter(ecef_target_grid[1,:],ecef_target_grid[2,:],ecef_target_grid[3,:], markersize=0.5)


# Define target valume based on C,H (from SCH) and azimuth angle

In [ ]:
# target volume grid on surface defined in ch (of sch) and azimuth rotation angles (rotated around platform position vector)
t_c  = 1e6:5e5:5e6 # m target positions in c of sch
t_h  = 0:1e5:5e5 # m target heights
rot_P=-20:.5:20#-60:10:60 # deg rotation angles around platform position vector

#define platform lat,lon,hgt
platform_geo = [60,-30,500e3] #(deg, deg, m)

#create Peg point
γ   = 0 # deg track angle (heading), 0 deg is north, 90 deg is east
peg = Geometry.PegPoint(platform_geo[1],platform_geo[2],γ) # peg point is the nadir point of platform at the center of SAR aperture

#create XYZ grid based on C,H,Az
t_xyz_grid_rot=Scene.chP_to_xyz_grid(t_c,t_h,rot_P,platform_geo, peg)# convert target volume from geo to xyz

#plot the ECEF grid
scatter(t_xyz_grid_rot[1,:],t_xyz_grid_rot[2,:],t_xyz_grid_rot[3,:],camera=(30,30),markersize=1)
